# AI Companion with Memory

This Jupyter notebook implements an AI companion that engages in conversations and remembers previous interactions using OpenAI's GPT-4 model and a memory system.

## Overview

The AI Companion utilizes the following components:
- OpenAI's GPT-4 for natural language processing
- mem0ai for memory management
- Qdrant for vector storage
- Neo4j for graph storage

### Notes

*1. Ensure all necessary credentials and API keys are properly set up before running the notebook. Do not share sensitive information when pushing to a public repository.*

*2. This notebook is designed to run in Google Colab. Ensure that you have set up the necessary credentials in Google Colab's userdata before running the notebook.*

## Installing mem0ai and other dependencies

In [1]:
!pip install mem0ai openai

# Importing Libraries

In [11]:
import os
import json
from google.colab import userdata
from openai import OpenAI
from mem0 import Memory
from typing import List, Dict

## Setting the OpenAI API keys

In [12]:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

## Setting Credentials

In [13]:
URL = userdata.get('URL')
USERNAME = userdata.get('USERNAME')
PASSWORD = userdata.get('PASSWORD')
QDRANT_COLLECTION = userdata.get('QDRANT_COLLECTION')
QDRANT_URL = userdata.get('QDRANT_URL')
QDRANT_API_KEY = userdata.get('QDRANT_API_KEY')
user_id=userdata.get('USER_ID')

# Setting Configurations

In [14]:
# Initialize the OpenAI client
client = OpenAI()

config = {
            "llm": {
                "provider": "openai",
                "config": {
                    "model": "gpt-4o",
                    "temperature": 0.2,
                }
            },
            "vector_store": {
        "provider": "qdrant",
        "config": {
            "collection_name": QDRANT_COLLECTION,
            "url": QDRANT_URL,
            "api_key": QDRANT_API_KEY
        }
    },
            "graph_store": {
                "provider": "neo4j",
                "config": {
                    "url": URL,
                    "username": USERNAME,
                    "password": PASSWORD
                },
            },
            "version": "v1.1"
        }

memory = Memory.from_config(config_dict=config)

**Checking the Memory**

In [ ]:
memory.get_all(user_id=user_id)

# AI Assistant

In [17]:
class Companion:
    """
    A class representing an AI companion that can engage in conversations and remember previous interactions.
    """

    def __init__(self, client: OpenAI):
        """
        Initialize the Companion.

        :param client: An instance of the OpenAI client for API interactions.
        """
        self.client = client

    def ask(self, question: str, user_id: str) -> str:
        """
        Process a user's question and generate a response.

        :param question: The user's input question.
        :param user_id: The unique identifier for the user.
        :return: The AI-generated response to the question.
        """
        # Retrieve relevant previous memories
        previous_memories = memory.search(question, user_id=user_id)
        relevant_memories_text = "\n".join(mem["memory"] for mem in previous_memories['results'])

        # Construct the prompt with the question and relevant memories
        prompt = f"User input: {question}\nPrevious memories: {relevant_memories_text}"
        messages = [
            {
                "role": "system",
                "content": "You are the user's companion. Use the user's input and previous memories to respond. Answer based on the context provided."
            },
            {
                "role": "user",
                "content": prompt
            }
        ]

        try:
            # Generate a response using the OpenAI API
            stream = self.client.chat.completions.create(
                model="gpt-4",
                stream=True,
                messages=messages
            )

            answer = ""
            for chunk in stream:
                if chunk.choices[0].delta.content is not None:
                    content = chunk.choices[0].delta.content
                    print(content, end="", flush=True)
                    answer += content

            # Add the new interaction to memory
            memory.add(question, user_id=user_id)
            return answer
        except Exception as e:
            print(f"An error occurred: {e}")
            return ""

    def __enter__(self):
        """
        Enter the runtime context for the Companion.

        :return: The Companion instance.
        """
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        """
        Exit the runtime context for the Companion.

        :param exc_type: The exception type, if any.
        :param exc_val: The exception value, if any.
        :param exc_tb: The exception traceback, if any.
        """
        pass

def main():
    """
    The main function to run the AI companion interaction loop.
    """
    with Companion(client) as ai_companion:
        while True:
            text_input = input("\nEnter text (or 'quit' to exit):\t")
            if text_input.lower() == 'quit':
                print("Goodbye :)")
                break
            print("\nAssistant:")
            ai_companion.ask(text_input, user_id)

if __name__ == "__main__":
    main()


Enter text (or 'quit' to exit):	Hi

Assistant:
Hello James! How can I assist you today?
Enter text (or 'quit' to exit):	quit
Goodbye :)
